In [27]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
import pprint

In [28]:
client = MongoClient('localhost', 27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [29]:
db = client.zams
collection = db.employees
print(f"Accessed Databse: {db.name}")
print(f"Accessed Collection: {collection.name}")

Accessed Databse: zams
Accessed Collection: employees


In [30]:
employees = collection.find()
# for employee in employees:
#     pprint.pprint(employee)

In [31]:
start_pay = '01-28-2024'
end_pay = '03-28-2024'

In [32]:
result =[]
for employee in employees:
    First_Name = employee['first_name']
    Last_Name = employee['last_name']
    salaries = employee['salaries']
    Employee_ID = employee['_id']
    Department = employee['department']['department_name']
    
    sum_gross_pay = 0
    sum_tax_paid = 0
    salary_count = 0
    sum_net_pay = 0
    
    gross_pay_01 = None
    gross_pay_03 = None
    
    for salary in salaries:
        sum_gross_pay += salary['gross_pay']
        sum_tax_paid += salary['tax_deduction']
        salary_count += 1 
        net = salary['gross_pay'] - (salary['tax_deduction'] + salary['social_security_deduction'])
        sum_net_pay += net
        
    avg_gross_wage = sum_gross_pay / salary_count if salary_count != 0 else 0
    avg_tax_paid = sum_tax_paid / salary_count if salary_count != 0 else 0
    avg_net_wage = sum_net_pay /salary_count if salary_count !=0 else 0
    
    
    result.append({
        'Employee_ID' : Employee_ID,
        'First_Name' : First_Name,
        'Last_Name' : Last_Name,
        'Department' : Department,
        'Avg_gross_wage' : np.round(avg_gross_wage,2),
        'Avg_Tax_Paid' : np.round(avg_tax_paid,2),
        'Avg_Net_Wage' : np.round(avg_net_wage,2),
          
    })

query = pd.DataFrame(result)
query.sort_values('Employee_ID')
    

,Employee_ID,First_Name,Last_Name,Department,Avg_gross_wage,Avg_Tax_Paid,Avg_Net_Wage
0,1,Jackie,Chan,Legal,12183.33,4069.23,7358.73
1,2,ikun,Cai,Marketing,12388.89,3840.55,7780.22
2,3,Dennis,Hancock,Sales,12648.61,2592.97,9271.43
3,4,Jennifer,Gibson,Design,15266.67,3099.13,11221.01
4,5,Angela,Ellison,Marketing,9479.17,2407.71,6483.75
5,6,Aaron,Kemp,Finance,8500.00,2380.00,5593.00
6,7,Daniel,Farley,Finance,10312.50,1959.37,7713.75
7,8,Barbara,Adams,Legal,12100.00,4755.30,6594.50
8,9,Nicholas,Mckinney,Finance,15405.56,4221.12,10229.29
9,10,Joshua,King,Legal,8644.44,1780.76,6327.73


In [33]:
pipeline = [
    {
        "$unwind": {
            "path": "$salaries",
            "preserveNullAndEmptyArrays": False
        }
    },
    {
        "$group": {
            "_id": "$department.department_name",
            "Employees_Working_Overtime": {
                "$addToSet": {
                    "$cond": [
                        {"$gt": ["$salaries.total_work_time", 160]},
                        "$_id",
                        None
                    ]
                }
            },
            "Total_Employees": {
                "$addToSet": "$_id"
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Depart_Name": "$_id",
            "Employees_Working_Overtime": {
                "$size": {
                    "$filter": {
                        "input": "$Employees_Working_Overtime",
                        "as": "id",
                        "cond": {"$ne": ["$$id", None]}
                    }
                }
            },
            "Total_Employees": {"$size": "$Total_Employees"}
        }
    }
]
query5 = collection.aggregate(pipeline)
for result in query5:
    print(result)





{'Depart_Name': 'Facilities', 'Employees_Working_Overtime': 3, 'Total_Employees': 6}
{'Depart_Name': 'Engineering and Technology', 'Employees_Working_Overtime': 3, 'Total_Employees': 4}
{'Depart_Name': 'Sales', 'Employees_Working_Overtime': 2, 'Total_Employees': 4}
{'Depart_Name': 'Legal', 'Employees_Working_Overtime': 5, 'Total_Employees': 6}
{'Depart_Name': 'Communications', 'Employees_Working_Overtime': 1, 'Total_Employees': 5}
{'Depart_Name': 'Finance', 'Employees_Working_Overtime': 5, 'Total_Employees': 7}
{'Depart_Name': 'Design', 'Employees_Working_Overtime': 4, 'Total_Employees': 6}
{'Depart_Name': 'Marketing', 'Employees_Working_Overtime': 3, 'Total_Employees': 6}
{'Depart_Name': 'Service and Support', 'Employees_Working_Overtime': 3, 'Total_Employees': 6}
